We can find the data of character interaction networks for George R. R. Martin’s “[A Song of Ice and Fire](https://github.com/mathbeveridge/asoiaf)”.

In [1]:
import pandas as pd
nodes = pd.read_csv('ASOIAF_nodes.csv')
nodes = nodes['Id']
num_nodes = nodes.shape[0]
nodes

0                        Addam-Marbrand
1           Aegon-Frey-(son-of-Stevron)
2                     Aegon-I-Targaryen
3      Aegon-Targaryen-(son-of-Rhaegar)
4                     Aegon-V-Targaryen
                     ...               
791                         Yorko-Terys
792                              Ysilla
793                   Yurkhaz-zo-Yunzak
794                                 Zei
795                               Zollo
Name: Id, Length: 796, dtype: object

Cool! So we have a list of characters in the variable node. There is a total of 795 nodes (characters). Now let’s explore edges.

In [2]:
edges = pd.read_csv('ASOIAF_edges.csv',usecols=[0,1] )
edges

,Source,Target
0,Addam-Marbrand,Brynden-Tully
1,Addam-Marbrand,Cersei-Lannister
2,Addam-Marbrand,Gyles-Rosby
3,Addam-Marbrand,Jaime-Lannister
4,Addam-Marbrand,Jalabhar-Xho
...,...,...
2818,Walder-Frey-(son-of-Merrett),Wex-Pyke
2819,Waymar-Royce,Will-(prologue)
2820,Weasel,Weese
2821,Woth,Yoren


Visualize Graph with Graphviz
Graphviz is a Python interface for drawing graphs.

In [3]:
!pip install graphviz

In [4]:
from graphviz import Digraph
dot = Digraph(comment='VIP graph')

Let’s visualize the first ten nodes

In [5]:
nodes_G = []
for node in edges[:10]['Source']:
    if node not in nodes_G:
        nodes_G.append(node)
for node in edges[:10]['Target']:
    if node not in nodes_G:
        nodes_G.append(node)

Add nodes and edges into the graph

In [6]:
dot = Digraph(comment='VIP graph')
for i in range(len(nodes_G)):
    dot.node(nodes_G[i])
for i in range(len(edges[:10])):
    edge = edges.iloc[i]
    dot.edge(edge['Source'], edge['Target'])

In [7]:
dot.render('VIP-graph_10.gv', view=True)

'VIP-graph_10.gv.pdf'

Nice! We can see the interactions of Addam-Marbrand clearly in the graph. But how about visualizing the entire network. Of course, we can do that. But we should anticipate that the network of characters in 5 chapters of this series would be huge

In [8]:
dot = Digraph(comment='VIP graph')
for i in range(num_nodes):
    dot.node(nodes[i])
for i in range(len(edges)):
    edge = edges.iloc[i]
    dot.edge(edge['Source'], edge['Target'])
dot.render('VIP-graph.gv', view=True)

'VIP-graph.gv.pdf'

Cool! With graph, we can visualize the network efficiently. But can we create our own graph class?

Start with creating the vertex (node) class. What attributes and methods do we want our vertex class to have? Hmm. Add connections (*addNeighbor*), know which nodes the current node connect with (*getConnections*), find out the name of the node (*getId* ), and how strong is the connection (*getWeight*)

In [9]:
## The Vertex

class Vertex:
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}
    
    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight

    def __str__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])

    def getConnections(self):
        return self.connectedTo.keys()

    def getId(self):
        return self.id

    def getWeight(self,nbr):
        return self.connectedTo[nbr]

Create a graph class with some basic methods of a graph:

In [10]:
class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self,key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self,n):
        return n in self.vertList

    def addEdge(self,f,t,cost=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], cost)

    def getVertices(self):
        return self.vertlist.keys()

    def __iter__(self):
        return iter(self.vertList.values())

In [11]:
g = Graph()

In [12]:
for i in range(num_nodes):
    g.addVertex(nodes[i])

In [13]:
for i in range(len(edges)):
    edge = edges.iloc[i]
    g.addEdge(edge['Source'], edge['Target'])
    g.addEdge(edge['Target'], edge['Source'])

In [15]:
for v in g:
    for w in v.getConnections():
        print(" %s<->%s" % (v.getId(), w.getId() ) )

Streaming output truncated to the last 5000 lines.
 Boros-Blount<->Meryn-Trant
 Boros-Blount<->Osmund-Kettleblack
 Boros-Blount<->Osney-Kettleblack
 Boros-Blount<->Sansa-Stark
 Boros-Blount<->Tanda-Stokeworth
 Boros-Blount<->Tommen-Baratheon
 Boros-Blount<->Tyrion-Lannister
 Bowen-Marsh<->Aemon-Targaryen-(Maester-Aemon)
 Bowen-Marsh<->Alliser-Thorne
 Bowen-Marsh<->Cellador
 Bowen-Marsh<->Cotter-Pyke
 Bowen-Marsh<->Denys-Mallister
 Bowen-Marsh<->Donal-Noye
 Bowen-Marsh<->Eddison-Tollett
 Bowen-Marsh<->Hobb
 Bowen-Marsh<->Janos-Slynt
 Bowen-Marsh<->Jeor-Mormont
 Bowen-Marsh<->Jon-Snow
 Bowen-Marsh<->Mance-Rayder
 Bowen-Marsh<->Othell-Yarwyck
 Bowen-Marsh<->Samwell-Tarly
 Bowen-Marsh<->Satin
 Bowen-Marsh<->Stannis-Baratheon
 Bowen-Marsh<->Tormund
 Bowen-Marsh<->Wick-Whittlestick
 Bran-Stark<->Alebelly
 Bran-Stark<->Arya-Stark
 Bran-Stark<->Benjen-Stark
 Bran-Stark<->Beth-Cassel
 Bran-Stark<->Brynden-Rivers
 Bran-Stark<->Catelyn-Stark
 Bran-Stark<->Cersei-Lannister
 Bran-Stark<->Chayle
 Br

In [16]:
print(g.getVertex('Addam-Marbrand').getConnections())

dict_keys([<__main__.Vertex object at 0x7fb4629ce350>, <__main__.Vertex object at 0x7fb46f1af510>, <__main__.Vertex object at 0x7fb4629c3410>, <__main__.Vertex object at 0x7fb46231b610>, <__main__.Vertex object at 0x7fb46231b650>, <__main__.Vertex object at 0x7fb46231ba90>, <__main__.Vertex object at 0x7fb462322190>, <__main__.Vertex object at 0x7fb462322bd0>, <__main__.Vertex object at 0x7fb4622b0250>, <__main__.Vertex object at 0x7fb4622c6910>, <__main__.Vertex object at 0x7fb4622c69d0>, <__main__.Vertex object at 0x7fb4622c6d10>])


Traverse the Nodes with Depth First Search

In [17]:
def traverseAll(g):
    
    visited = []
    connected = 0
    for v in g:
        if v not in visited:
            visited_v = dfs(v)
            visited += visited_v
            connected += 1
            print('\n\nNode {} has {} connected components'.format(v.getId(), len(visited_v)))
            print("\n\n")
    
    print("Total number of connected components is {}".format(connected))

In [18]:
def dfs(s):
    stack = [s]
    visited = []
    while stack:
        v = stack.pop()
        if v not in visited:
            visited.append(v)
            print (v.getId(), end = ", ") 
            for w in v.getConnections():
                if w not in visited:
                    stack.append(w)
                
    #print(len(visited))
    return visited

In [19]:
traverseAll(g)

Addam-Marbrand, Varys, Tywin-Lannister, Wylis-Manderly, Jaime-Lannister, Zollo, Vargo-Hoat, Rorge, Yoren, Woth, Lommy-Greenhands, Weasel, Weese, Polliver, Tickler, Sandor-Clegane, Tyrion-Lannister, Ysilla, Yandry, Rolly-Duckfield, Lemore, Jon-Connington, Robert-Baratheon, Willam-Dustin, Eddard-Stark, Wyman-Manderly, Theon-Greyjoy, Yellow-Dick, Sour-Alyn, Skinner, Ramsay-Snow, Walder-Frey-(son-of-Merrett), Wex-Pyke, Lorren, Urzen, Sybelle-Glover, Asha-Greyjoy, Victarion-Greyjoy, Wulfe, Talbert-Serry, Rodrik-Sparr, Moqorro, Daenerys-Targaryen, Yurkhaz-zo-Yunzak, Xaro-Xhoan-Daxos, Pyat-Pree, Barristan-Selmy, Widower, Steelskin, Khrazz, Spotted-Cat, Hizdahr-zo-Loraq, Skahaz-mo-Kandaq, Reznak-mo-Reznak, Galazza-Galare, Grazhar, Qezza, Miklaz, Draqaz, Mezzara, Missandei, Quentyn-Martell, Trystane-Martell, Myrcella-Baratheon, Tommen-Baratheon, Taena-of-Myr, Osney-Kettleblack, Stannis-Baratheon, William-Foxglove, Val, Tormund, Ygritte, Styr, Rattleshirt, Weeper, Othell-Yarwyck, Pypar, Todder, 

Breadth First Search to Find the Distance Between Characters

In [20]:
def bfs(g,s,t):
    visited = {}
    d = {}
    for v in g:
        visited[v] = False
        d[v] = float('inf')
     
    d[s] = 0
    queue = [s]
    visited[s] = True
    
    while queue:
        v = queue.pop(0)
        
        for w in v.getConnections():
            if visited[w] == False:
                visited[w] = True
                queue.append(w)
                d[w] = d[v] + 1
    print("The distance from {} to {} is {}".format(s.getId(),t.getId(), d[t]))

In [21]:
s = g.getVertex('Addam-Marbrand')
t = g.getVertex('Emmond')
bfs(g,s,t)

The distance from Addam-Marbrand to Emmond is 4


In [22]:
s = g.getVertex('Shitmouth')
t = g.getVertex('Trystane-Martell')
bfs(g,s,t)

The distance from Shitmouth to Trystane-Martell is 4
